In [ ]:
import requests
import time

REFRESH_TOKEN = ""

ARTISTS = [
    {"label": "Ed Sheeran", "id": 3648},
    {"label": "The Weeknd", "id": 3852},
    {"label": "Tones And I", "id": 1410939},
    {"label": "Imagine Dragons", "id": 4132},
    {"label": "Harry Styles", "id": 558681},
    {"label": "Mark Ronson", "id": 1759},
    {"label": "Bruno Mars", "id": 3501},
    {"label": "Luis Fonsi", "id": 1434},
    {"label": "Daddy Yankee", "id": 1802},
    {"label": "Justin Bieber", "id": 3479},
    {"label": "The Kid LAROI", "id": 1333786},
    {"label": "Glass Animals", "id": 3656},
    {"label": "The Chainsmokers", "id": 4796},
    {"label": "Halsey", "id": 5109},
    {"label": "Miley Cyrus", "id": 2619},
    {"label": "Lewis Capaldi", "id": 550208},
    {"label": "Daft Punk", "id": 170},
    {"label": "Taylor Swift", "id": 2762},
    {"label": "Dua Lipa", "id": 5381},
    {"label": "Drake", "id": 3380},
    {"label": "Christina Perri", "id": 3684},
    {"label": "OneRepublic", "id": 2796},
    {"label": "Juice WRLD", "id": 921966},
    {"label": "Justin Beiber", "id": 3476},
    {"label": "Coldplay", "id": 439},
    {"label": "Lady Gaga", "id": 3188},
    {"label": "Shawn Mendes", "id": 4934},
    {"label": "Camila Cabello", "id": 454302},
    {"label": "Pritam", "id": 127583},
    {"label": "Arijit Singh", "id": 303406},
    {"label": "Shreya Ghoshal", "id": 23385},
    {"label": "Rajat Nagpal", "id": 1349325},
    {"label": "Jubin Nautiyal", "id": 785271},
    {"label": "Tanishk Bagchi", "id": 388884},
    {"label": "Dhvani Bhanushali", "id": 874442},
    {"label": "Alka Yagnik", "id": 267405},
    {"label": "Sonu Nigam", "id": 108808},
    {"label": "Phora", "id": 206715},
    {"label": "M.I.A.", "id": 1537},
    {"label": "Metric", "id": 122408},
    {"label": "Cigarettes After Sex", "id": 4483},
    {"label": "YG", "id": 4252},
    {"label": "JAY-Z", "id": 58},
    {"label": "Sam Smith", "id": 3176},
    {"label": "Franz Ferdinand", "id": 1794},
    {"label": "Sigur Rós", "id": 755}
]

def refresh_chartmetric_access_token():
    url = "https://api.chartmetric.com/api/token"
    headers = {"Content-Type": "application/json"}
    data = {"refreshtoken": REFRESH_TOKEN}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json().get('token')
    else:
        print("Error refreshing token:", response.text)
        return None

def get_artist_career(artist_id, access_token):
    url = f"https://api.chartmetric.com/api/artist/{artist_id}/career"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"limit": 1}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data and 'obj' in data and len(data['obj']) > 0:
            return data['obj'][0]  # Use the first (latest) entry
    else:
        print(f"Error fetching career for artist {artist_id}: {response.text}")
    return None


def build_artist_career_list():
    access_token = refresh_chartmetric_access_token()
    if not access_token:
        return []
    artist_careers = []
    for artist in ARTISTS:
        career = get_artist_career(artist["id"], access_token)
        if career:
            artist_careers.append({
                "label": artist["label"],
                "id": artist["id"],
                "stage": career.get("stage"),
                "momentum": career.get("momentum"),
                "stage_score": career.get("stage_score"),
                "momentum_score": career.get("momentum_score"),
            })
        time.sleep(0.2)  # To avoid rate limits
    return artist_careers

def main():
    artist_careers = build_artist_career_list()
    if not artist_careers:
        print("No artist data found.")
        return

    # Extract unique stages and momentum categories
    stages = sorted(set(a['stage'] for a in artist_careers if a['stage']))
    momentums = sorted(set(a['momentum'] for a in artist_careers if a['momentum']))

    while True:
        print("\nSort/filter by:")
        print("1. Stage")
        print("2. Momentum")
        print("3. Exit")
        choice = input("Enter your choice (1/2/3): ").strip()
        if choice == "1":
            print("\nAvailable stages:")
            for idx, s in enumerate(stages, 1):
                print(f"{idx}. {s}")
            val = input("Select stage by number or type 'all': ").strip()
            if val == "all":
                filtered = artist_careers
            elif val.isdigit() and 1 <= int(val) <= len(stages):
                selected = stages[int(val) - 1]
                filtered = [a for a in artist_careers if a['stage'] == selected]
            else:
                print("Invalid input.")
                continue
        elif choice == "2":
            print("\nAvailable momentum categories:")
            for idx, m in enumerate(momentums, 1):
                print(f"{idx}. {m}")
            val = input("Select momentum by number or type 'all': ").strip()
            if val == "all":
                filtered = artist_careers
            elif val.isdigit() and 1 <= int(val) <= len(momentums):
                selected = momentums[int(val) - 1]
                filtered = [a for a in artist_careers if a['momentum'] == selected]
            else:
                print("Invalid input.")
                continue
        elif choice == "3":
            print("Exiting.")
            break
        else:
            print("Invalid choice.")
            continue

        print("\nMatching Artists:")
        for a in sorted(filtered, key=lambda x: x['label']):
            print(f"{a['label']} (ID: {a['id']}), Stage: {a['stage']}, Momentum: {a['momentum']}")
        print("-" * 40)

if __name__ == "__main__":
    main()



Sort/filter by:
1. Stage
2. Momentum
3. Exit


Enter your choice (1/2/3):  1



Available stages:
1. legendary
2. mainstream
3. superstar


Select stage by number or type 'all':  2



Matching Artists:
Metric (ID: 122408), Stage: mainstream, Momentum: gradual decline
Phora (ID: 206715), Stage: mainstream, Momentum: gradual decline
Rajat Nagpal (ID: 1349325), Stage: mainstream, Momentum: gradual decline
Sigur Rós (ID: 755), Stage: mainstream, Momentum: decline
----------------------------------------

Sort/filter by:
1. Stage
2. Momentum
3. Exit


...


Matching Artists:
Alka Yagnik (ID: 267405), Stage: legendary, Momentum: steady
Arijit Singh (ID: 303406), Stage: superstar, Momentum: steady
Bruno Mars (ID: 3501), Stage: superstar, Momentum: steady
Christina Perri (ID: 3684), Stage: superstar, Momentum: steady
Daddy Yankee (ID: 1802), Stage: superstar, Momentum: steady
Daft Punk (ID: 170), Stage: superstar, Momentum: steady
Drake (ID: 3380), Stage: superstar, Momentum: steady
Dua Lipa (ID: 5381), Stage: superstar, Momentum: steady
Franz Ferdinand (ID: 1794), Stage: superstar, Momentum: steady
Halsey (ID: 5109), Stage: superstar, Momentum: steady
Harry Styles (ID: 558681), Stage: superstar, Momentum: steady
JAY-Z (ID: 58), Stage: superstar, Momentum: steady
Juice WRLD (ID: 921966), Stage: superstar, Momentum: steady
Justin Bieber (ID: 3479), Stage: superstar, Momentum: steady
Luis Fonsi (ID: 1434), Stage: legendary, Momentum: steady
M.I.A. (ID: 1537), Stage: superstar, Momentum: steady
Miley Cyrus (ID: 2619), Stage: superstar, Momentu

Enter your choice (1/2/3):  1



Available stages:
1. legendary
2. mainstream
3. superstar


Select stage by number or type 'all':  3



Matching Artists:
Arijit Singh (ID: 303406), Stage: superstar, Momentum: steady
Bruno Mars (ID: 3501), Stage: superstar, Momentum: steady
Camila Cabello (ID: 454302), Stage: superstar, Momentum: gradual decline
Christina Perri (ID: 3684), Stage: superstar, Momentum: steady
Cigarettes After Sex (ID: 4483), Stage: superstar, Momentum: gradual decline
Coldplay (ID: 439), Stage: superstar, Momentum: gradual decline
Daddy Yankee (ID: 1802), Stage: superstar, Momentum: steady
Daft Punk (ID: 170), Stage: superstar, Momentum: steady
Dhvani Bhanushali (ID: 874442), Stage: superstar, Momentum: decline
Drake (ID: 3380), Stage: superstar, Momentum: steady
Dua Lipa (ID: 5381), Stage: superstar, Momentum: steady
Ed Sheeran (ID: 3648), Stage: superstar, Momentum: growth
Franz Ferdinand (ID: 1794), Stage: superstar, Momentum: steady
Glass Animals (ID: 3656), Stage: superstar, Momentum: gradual decline
Halsey (ID: 5109), Stage: superstar, Momentum: steady
Harry Styles (ID: 558681), Stage: superstar, M